In [14]:
import time
from collections import deque
from optibook.synchronous_client import Exchange
import logging

logger = logging.getLogger('client')
logger.setLevel('INFO')

exchange = Exchange()
exchange.connect()

In [37]:
positions = {}

In [10]:
# Define the specific pairs of original and dual-listed stocks
stock_pairs = [
    ('NVDA', 'NVDA_DUAL'),
    ('SAN', 'SAN_DUAL')
]

tradable_assets = ['NVDA','NVDA_DUAL','SAN','SAN_DUAL']

In [15]:
# Global Variables
price_history = {}
for asset in tradable_assets:
    price_history[asset] = {'bid':deque([1] * 50, maxlen=10), 'ask':deque([1] * 100, maxlen=10)}
print(price_history)

{'NVDA': {'bid': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10), 'ask': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)}, 'NVDA_DUAL': {'bid': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10), 'ask': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)}, 'SAN': {'bid': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10), 'ask': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)}, 'SAN_DUAL': {'bid': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10), 'ask': deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=10)}}


In [12]:
# # Initialize dictionary to track positions dynamically
# def initialize_positions():
#     for original_stock, dual_stock in stock_pairs:
#         positions[dual_stock] = exchange.get_positions()[dual_stock]
#         positions[original_stock] = exchange.get_positions()[dual_stock]
#     return positions

In [35]:
def get_best_prices(stock):
    order_book = exchange.get_last_price_book(stock)
    if not order_book.bids:
        best_bid = price_history[stock]['bid'][0]
        price_history[stock]['bid'].appendleft(best_bid)
    else:
        price_history[stock]['bid'].appendleft(order_book.bids[0].price)
        best_bid = round(sum(price_history[stock]['bid'])/10, 1)
        
    if not order_book.asks:
        best_ask = price_history[stock]['ask'][0]
        price_history[stock]['ask'].appendleft(best_ask)
    else:
        price_history[stock]['ask'].appendleft(order_book.asks[0].price)
        best_ask = round(sum(price_history[stock]['ask'])/10, 1)
    
    if best_bid - best_ask >= 0.3:
        best_bid += 0.1
        best_ask -= 0.1      
    return {"bid": best_bid, "ask": best_ask}

In [34]:
def adjustBidAskForHedge(stock, bidAskDict):
    position = exchange.get_positions()[stock]
    if position > 0:
        bidAskDict["ask"] -= round(position * 0.002, 1)
    elif position < 0:
        bidAskDict["bid"] -= round(position * 0.002, 1)
    return bidAskDict

In [ ]:
def checkValidBidAsk(stock, bidAskDict):
    position = exchange.get_positions()[stock]
    spread = bidAskDict['ask'] - bidAskDict['bid']
    if spread < 0:
        if position > 0:
            bidAskDict["bid"] = bidAskDict["ask"] - 0.10
        if position <= 0:
            bidAskDict["ask"] = bidAskDict["bid"] + 0.10
    return bidAskDict

In [7]:
def checkSpreadOfOrig(stock, bidAskDict):
    order_book = exchange.get_last_price_book(instrument_id=stock)
    if order_book.asks:
        while bidAskDict["ask"] <= order_book.asks[0].price:
            bidAskDict["ask"] += 0.1
    if order_book.bids:
        while bidAskDict["bid"] >= order_book.bids[0].price:
            bidAskDict["bid"] -= 0.1       
    return bidAskDict

In [39]:
# Hedge the underlying position
def hedge(stock_pair):
    positions = exchange.get_positions()
    original_stock, dual_stock = stock_pair
    positionTotal = positions[original_stock] + positions[dual_stock]

    if positionTotal > 0:
        order_book = exchange.get_last_price_book(instrument_id=original_stock)
        if order_book.bids:
            bestPrice = order_book.bids[0].price
        if bestPrice:
            placeOrder(original_stock, bestPrice, positionTotal, False, order_type="ioc")
                
    if positionTotal < 0:
        order_book = exchange.get_last_price_book(instrument_id=original_stock)
        if order_book.asks:
            bestPrice = order_book.asks[0].price
        if bestPrice:
            placeOrder(original_stock, bestPrice, positionTotal, True, order_type="ioc")

In [33]:
def placeOrder(stock, price, quantity, bid, orderType):
    if bid:
        exchange.insert_order(stock, price=price, volume=quantity, side='bid', order_type=orderType)
    else:
        exchange.insert_order(stock, price=price, volume=quantity, side='ask', order_type=orderType)

In [32]:
def deleteOutstandingOrders(stock):
    outstanding = exchange.get_outstanding_orders(stock)
    for order in outstanding.values():
        exchange.delete(stock, order_id=order.order_id)

In [ ]:
def trade():
    while True:
    for stock_pair in stock_pairs:
        original_stock, dual_stock = stock_pair
        deleteOutstandingOrders(dual_stock)
        
        # find the best bid and ask price
        bidAskDict = get_best_prices(dual_stock)
        bidAskDict = adjustBidAskForHedge(dual_stock, bidAskDict)
        bidAskDict = checkValidBidAsk(dual_stock, bidAskDict)
        bidAskDict = checkSpreadOfOrig(original_stock)
        
        positions = exchange.get_positions()
        current_position = positions[dual_stock]
        bid_volume = max(0, 100 - current_position) * 0.5
        ask_volume = max(0, current_position + 100) * 0.5
        
        if bid_volume > 0:
            placeOrder(dual_stock, bidAskDict['bid'], bid_volume, True, "limit")

        if ask_volume > 0:
            placeOrder(dual_stock, bidAskDict['ask'], ask_volume, False, "limit")
        
        # hedge the position
        hedge(stock_pair)
        
        logger.info(positions)
        time.sleep(0.05)
        
        